In [89]:
import pandas as pd
import numpy as numpy
import json 
import os 
import pylab as pl
import urllib
import requests
import geopandas as gp
import fiona
import shapely
from fiona.crs import from_epsg
import os
import pandas as pd
import urllib
import requests
import json
import sys
import io
import shapely
import base64
import urllib.request
import functools
import geopandas as gp
import fiona
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
%matplotlib inline

%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/katharinevoorhees/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['nan']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [90]:
myAPI = <<your api>>

In [91]:
#pull in library of variable names
url = "https://api.census.gov/data/2016/acs/acs5/variables.json"
resp = requests.request('GET', url)
aff1y = json.loads(resp.text)

affkeys = np.array(list(aff1y['variables'].keys()))

In [92]:
#search for commute codes
[(k, aff1y['variables'][k]['label'])  for k in affkeys if k.startswith ('B08301') ]

[('B08301_001E', 'Estimate!!Total'),
 ('B08301_010E', 'Estimate!!Total!!Public transportation (excluding taxicab)'),
 ('B08301_011E',
  'Estimate!!Total!!Public transportation (excluding taxicab)!!Bus or trolley bus'),
 ('B08301_012E',
  'Estimate!!Total!!Public transportation (excluding taxicab)!!Streetcar or trolley car (carro publico in Puerto Rico)'),
 ('B08301_013E',
  'Estimate!!Total!!Public transportation (excluding taxicab)!!Subway or elevated'),
 ('B08301_003E', 'Estimate!!Total!!Car, truck, or van!!Drove alone'),
 ('B08301_002E', 'Estimate!!Total!!Car, truck, or van'),
 ('B08301_005E',
  'Estimate!!Total!!Car, truck, or van!!Carpooled!!In 2-person carpool'),
 ('B08301_004E', 'Estimate!!Total!!Car, truck, or van!!Carpooled'),
 ('B08301_007E',
  'Estimate!!Total!!Car, truck, or van!!Carpooled!!In 4-person carpool'),
 ('B08301_006E',
  'Estimate!!Total!!Car, truck, or van!!Carpooled!!In 3-person carpool'),
 ('B08301_009E',
  'Estimate!!Total!!Car, truck, or van!!Carpooled!!In 7

In [93]:
#dictionary for renaming later
column_keys= {'B08301_001E':'total_total','B08301_010E':'total_public_transpo',
        'B08301_011E':'bus','B08301_012E':'streetcar','B08301_013E':'subway',
        'B08301_002E': 'total_car_van_truck', 'B08301_003E':'car_vt_alone',
        'B08301_004E':'car_vt_carpooled', 'B08301_018E':'bicycle','B08301_019E':'walked',
        'B08301_014E':'railroad', 'B08301_015E':'ferry','B08301_016E':'taxi',
        'B08301_017E':'motorcycle','B08301_021E':'worked_at_home', 'B08301_020E':'other_means'}

In [94]:
#name keys
key1 = 'B08301_001E,B08301_010E,B08301_011E,B08301_012E,B08301_013E,B08301_002E,B08301_003E,'
key2 = 'B08301_004E,B08301_018E,B08301_019E,B08301_014E,B08301_015E,B08301_016E,B08301_017E,'
key3 = 'B08301_021E,B08301_020E'

In [95]:
#download info from census
year = '2015'

counties = [ 5, 47, 61, 81, 85]

url = "https://api.census.gov/data/" + year + "/acs/acs5?get=" +key1+key2+key3+\
    ",NAME&for=tract:*&in=state:36&in=county:*&key=" + myAPI
resp = requests.request('GET', url).content
commute = pd.read_csv(io.StringIO(resp.decode('utf-8').replace('[','').replace(']','')))

commute = commute[commute['county'].isin(counties)]


commute.head(1)

,B08301_001E,B08301_010E,B08301_011E,B08301_012E,B08301_013E,B08301_002E,B08301_003E,B08301_004E,B08301_018E,B08301_019E,B08301_014E,B08301_015E,B08301_016E,B08301_017E,B08301_021E,B08301_020E,NAME,state,county,tract,Unnamed: 20
0,1812,1268,51,8,1190,418,338,80,42,35,19,0,0,0,49,0,"Census Tract 245, Kings County, New York",36,47,24500,NaN


In [96]:
commute.rename(columns=column_keys, inplace=True)

In [97]:
commute.head(3)

,total_total,total_public_transpo,bus,streetcar,subway,total_car_van_truck,car_vt_alone,car_vt_carpooled,bicycle,walked,railroad,ferry,taxi,motorcycle,worked_at_home,other_means,NAME,state,county,tract,Unnamed: 20
0,1812,1268,51,8,1190,418,338,80,42,35,19,0,0,0,49,0,"Census Tract 245, Kings County, New York",36,47,24500,NaN
1,1263,629,51,0,578,499,389,110,0,112,0,0,0,0,23,0,"Census Tract 246, Kings County, New York",36,47,24600,NaN
2,1041,790,63,17,703,123,86,37,19,60,7,0,0,0,33,16,"Census Tract 247, Kings County, New York",36,47,24700,NaN


In [98]:
#pull in previously made key
zones = pd.read_csv('better_taxizone_censustract_key.csv')
zones.head()

,index_right,OBJECTID,zone,tractid,bcode,tractnum
0,1968,2.0,Jamaica Bay,36081107202,36081,107202
1,222,3.0,Allerton/Pelham Gardens,36005031000,36005,31000
2,224,3.0,Allerton/Pelham Gardens,36005031400,36005,31400
3,226,3.0,Allerton/Pelham Gardens,36005031800,36005,31800
4,225,3.0,Allerton/Pelham Gardens,36005031600,36005,31600


In [99]:
len(zones['tractnum'].unique())

1332

In [100]:
len(commute['tract'].unique())

1335

In [101]:
#get country codes for join
zones['bcode'] = zones['bcode'].astype('str')
zones['cnty'] = zones['bcode'].str[-3:].astype('int')
zones.head()

,index_right,OBJECTID,zone,tractid,bcode,tractnum,cnty
0,1968,2.0,Jamaica Bay,36081107202,36081,107202,81
1,222,3.0,Allerton/Pelham Gardens,36005031000,36005,31000,5
2,224,3.0,Allerton/Pelham Gardens,36005031400,36005,31400,5
3,226,3.0,Allerton/Pelham Gardens,36005031800,36005,31800,5
4,225,3.0,Allerton/Pelham Gardens,36005031600,36005,31600,5


In [116]:
joined = commute.merge(zones, left_on=['county','tract'], right_on=['cnty','tractnum'])

joined.shape

(2161, 28)

In [117]:
#see if any tracts don't have a taxi zone
nan = joined[joined['OBJECTID'].isnull()]
nan

,total_total,total_public_transpo,bus,streetcar,subway,total_car_van_truck,car_vt_alone,car_vt_carpooled,bicycle,walked,railroad,ferry,taxi,motorcycle,worked_at_home,other_means,NAME,state,county,tract,Unnamed: 20,index_right,OBJECTID,zone,tractid,bcode,tractnum,cnty


In [118]:
#aggregate by zone
commute_taxi_zone = joined.groupby('OBJECTID').sum().reset_index()

In [119]:
commute_taxi_zone.columns

Index(['OBJECTID', 'total_total', 'total_public_transpo', 'bus', 'streetcar',
       'subway', 'total_car_van_truck', 'car_vt_alone', 'car_vt_carpooled',
       'bicycle', 'walked', 'railroad', 'ferry', 'taxi', 'motorcycle',
       'worked_at_home', 'other_means', 'state', 'county', 'tract',
       'Unnamed: 20', 'index_right', 'tractid', 'tractnum', 'cnty'],
      dtype='object')

In [121]:
commute_taxi_zone = commute_taxi_zone.loc[:,'OBJECTID':'other_means']

In [125]:
commute_taxi_zone.rename(columns={'OBJECTID':'taxi_zone'}, inplace=True)

commute_taxi_zone.head(2)

,taxi_zone,total_total,total_public_transpo,bus,streetcar,subway,total_car_van_truck,car_vt_alone,car_vt_carpooled,bicycle,walked,railroad,ferry,taxi,motorcycle,worked_at_home,other_means
0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3.0,13605,5930,1492,41,4197,6450,5476,974,5,683,200,0,86,0,370,81


In [126]:
commute_taxi_zone.to_csv('acs_commute_data_by_taxi_zone.csv')